<a href="https://colab.research.google.com/github/JayellWolfe/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/DS7_unit2_Polynomial_Log_linear_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intermediate Linear Regression Practice

## Use a Linear Regression model to get the lowest RMSE possible on the following dataset:

[Dataset Folder](https://github.com/ryanleeallred/datasets/tree/master/Ames%20Housing%20Data)

[Raw CSV](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv)

## You model must include (at least):
- A log-transformed y variable
- Two polynomial features
- One interaction feature
- 10 other engineered features

What is the lowest Root-Mean-Squared Error that you are able to obtain? Share your best RMSEs in Slack!

Notes:

There may be some data cleaning that you need to do on some features of this dataset. Linear Regression will only accept numeric values and will not accept

Note* There may not be a clear candidate for an interaction term in this dataset. Include one anyway, sometimes it's a good practice for predictive modeling feature engineering in general. 

In [0]:
#imports 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [18]:
url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv'
ames = pd.read_csv(url)
print(ames.shape)
ames.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [19]:
#identify NaNs
null_sums = ames.isnull().sum()
has_nulls = set(null_sums[null_sums > 0].index)
numeric_columns = set(ames.select_dtypes('number').columns)
categorical_columns = set(ames.select_dtypes(exclude='number').columns)

numeric_with_nulls = has_nulls & numeric_columns
categorical_with_nulls = has_nulls & categorical_columns

print(f'Categorical_with_nulls:\n{categorical_with_nulls}')
print(f'\nNumeric_with_nulls:\n{numeric_with_nulls}')

Categorical_with_nulls:
{'BsmtFinType2', 'GarageCond', 'GarageType', 'BsmtExposure', 'MiscFeature', 'Fence', 'BsmtFinType1', 'BsmtCond', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'MasVnrType', 'BsmtQual', 'PoolQC', 'Alley', 'Electrical'}

Numeric_with_nulls:
{'GarageYrBlt', 'MasVnrArea', 'LotFrontage'}


In [0]:
# Fill NaNs
ames['GarageYrBlt'].fillna(ames['GarageYrBlt'].mean(), inplace=True)
ames['LotFrontage'].fillna(0, inplace=True)
ames['MasVnrArea'].fillna(0, inplace=True)

# replace NaNs with str
ames.fillna("filled", inplace=True)

In [22]:
#encode data

# seperate out the ordinal columns
ordinal_columns = ['ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
                     'HeatingQC','CentralAir','KitchenQual','FireplaceQu','GarageQual',
                     'GarageCond','PoolQC','Fence']

ames_ordinal = ames[ordinal_columns].copy()

# define each columns categories
ordinal_cats = [['Ex','Gd','TA','Fa','Po','filled'], # ExterQual
                ['Ex','Gd','TA','Fa','Po','filled'], # ExterCond
                ['Ex','Gd','TA','Fa','Po','filled'], # BsmtQual 
                ['Ex','Gd','TA','Fa','Po','filled'], # BsmtCond
                ['Gd','Av','Mn','No','filled'], # BsmtExposure
                ['Ex','Gd','TA','Fa','Po','filled'], # HeatingQC
                ['Y','N'], # CentralAir
                ['Ex','Gd','TA','Fa','Po','filled'], # KitchenQual
                ['Ex','Gd','TA','Fa','Po','filled'], # FireplaceQu
                ['Ex','Gd','TA','Fa','Po','filled'], # GarageQual
                ['Ex','Gd','TA','Fa','Po','filled'], # GarageCond
                ['Ex','Gd','TA','Fa','Po','filled'], # PoolQC
                ['GdPrv','MnPrv','GdWo','MnWw','filled']] # Fence

# fit the model 
enc = preprocessing.OrdinalEncoder(ordinal_cats)
ordinal_encoded = pd.DataFrame(enc.fit_transform(ames_ordinal),
                           columns=ames_ordinal.columns)

print(f'Shape of ordinal dataframe: {ordinal_encoded.shape}')
ordinal_encoded.head()

Shape of ordinal dataframe: (1460, 13)


,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC,Fence
0,1.0,2.0,1.0,2.0,3.0,0.0,0.0,1.0,5.0,2.0,2.0,5.0,4.0
1,2.0,2.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,5.0,4.0
2,1.0,2.0,1.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0,2.0,5.0,4.0
3,2.0,2.0,2.0,1.0,3.0,1.0,0.0,1.0,1.0,2.0,2.0,5.0,4.0
4,1.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,5.0,4.0


In [23]:
#one-hot

#seperate out non-ordinal 
ames_categorical = ames[list(categorical_columns - set(ordinal_columns))].copy()

# fit the model/create dataframe
enc = preprocessing.OneHotEncoder(sparse=False)
categorical_encoded = pd.DataFrame(enc.fit_transform(ames_categorical))

print(f'Shape of categorical dataframe: {categorical_encoded.shape}')
categorical_encoded.head()

Shape of categorical dataframe: (1460, 206)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
#seperate out numerical columns into their own DF
numerical = ames[list(numeric_columns)].copy().drop(columns=['Id','SalePrice'])
print(f'Shape of numerical dataframe: {numerical.shape}')
numerical.head()

Shape of numerical dataframe: (1460, 36)


,KitchenAbvGr,GarageYrBlt,PoolArea,1stFlrSF,GarageArea,HalfBath,TotRmsAbvGrd,Fireplaces,TotalBsmtSF,EnclosedPorch,ScreenPorch,YearBuilt,GarageCars,LotFrontage,BsmtHalfBath,YearRemodAdd,BsmtFinSF1,OverallCond,OpenPorchSF,FullBath,MiscVal,MasVnrArea,LotArea,3SsnPorch,MoSold,GrLivArea,BsmtFinSF2,MSSubClass,LowQualFinSF,BedroomAbvGr,YrSold,OverallQual,2ndFlrSF,WoodDeckSF,BsmtUnfSF,BsmtFullBath
0,1,2003.0,0,856,548,1,8,0,856,0,0,2003,2,65.0,0,2003,706,5,61,2,0,196.0,8450,0,2,1710,0,60,0,3,2008,7,854,0,150,1
1,1,1976.0,0,1262,460,0,6,1,1262,0,0,1976,2,80.0,1,1976,978,8,0,2,0,0.0,9600,0,5,1262,0,20,0,3,2007,6,0,298,284,0
2,1,2001.0,0,920,608,1,6,1,920,0,0,2001,2,68.0,0,2002,486,5,42,2,0,162.0,11250,0,9,1786,0,60,0,3,2008,7,866,0,434,1
3,1,1998.0,0,961,642,0,7,1,756,272,0,1915,3,60.0,0,1970,216,5,35,1,0,0.0,9550,0,2,1717,0,70,0,3,2006,7,756,0,540,1
4,1,2000.0,0,1145,836,1,9,1,1145,0,0,2000,3,84.0,0,2000,655,5,84,2,0,350.0,14260,0,12,2198,0,60,0,4,2008,8,1053,192,490,1


In [25]:
#create single DF w/ all variables used
ames_all = pd.concat([numerical, ordinal_encoded, categorical_encoded], axis=1)
print(f'Shape of final dataframe: {ames_all.shape}')

#include dependant variable, 2 forms 
price = ames['SalePrice']
price_log = np.log(ames['SalePrice'])

Shape of final dataframe: (1460, 255)


In [0]:
#start regression 
def regress_df(X, y, test_size=0.5):
    """
    Runs a linear regression and prints out stats
    
    X: array of variables
    y: array of results
    test_size: parameter for train_test_split
    """
    # Slit into test and train datasets
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.5, random_state=42)

    # fit model using train datasets
    model = LinearRegression()
    model.fit(X_train, Y_train)

    # Create new predictions using x_test
    y_pred = model.predict(X_test)

    # Measure Accuracy using y_test and y_pred
    RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
    R2 = r2_score(Y_test, y_pred)

    print('RMSE is {}'.format(RMSE))
    print('R^2 is {}'.format(R2))

    print("Coefficients: ", model.coef_[0])
    print("Intercepts: ", model.intercept_)

In [35]:
# Regressing to the ordinal variables = works
y = price_log
X = ordinal_encoded

regress_df(X, y)

RMSE is 0.2218395471940883
R^2 is 0.7048816026437641
Coefficients:  -0.13602501293892702
Intercepts:  12.804623573611803


In [36]:
# regress to numericals = works
y = price_log
X = numerical
regress_df(X, y)

RMSE is 0.15403676629806864
R^2 is 0.8577123753513348
Coefficients:  -0.040110863405415434
Intercepts:  16.669220623314878


In [37]:
#regress to one -hot cats = does not work !
y = price_log
X = categorical_encoded
regress_this(X, y)

RMSE is 184741191842.7759
R^2 is -2.046660584360335e+23
Coefficients:  1550921469.2032022
Intercepts:  -200346515915.94144


In [0]:
# This handy function courtesy of Carlos Gutierrez.
def dummyEncode(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    le = preprocessing.LabelEncoder()
    for feature in columnsToEncode:
        try:
            df[feature] = le.fit_transform(df[feature])
        except:
            print('Error encoding '+feature)
    return df

In [39]:
# Categoricals are now encoded as just regular numbers
new_cats = dummyEncode(ames_categorical)
new_cats.head()

,Heating,LotShape,GarageType,MiscFeature,Functional,MSZoning,GarageFinish,Street,SaleCondition,LandSlope,Condition2,Utilities,Foundation,Neighborhood,RoofStyle,SaleType,RoofMatl,Exterior2nd,MasVnrType,LandContour,Condition1,Electrical,LotConfig,BsmtFinType2,BldgType,PavedDrive,Exterior1st,HouseStyle,BsmtFinType1,Alley
0,1,3,1,4,6,3,1,1,4,0,2,0,2,5,1,8,1,13,1,3,2,4,4,5,0,2,12,5,2,2
1,1,3,1,4,6,3,1,1,4,0,2,0,1,24,1,8,1,8,2,3,1,4,2,5,0,2,8,2,0,2
2,1,0,1,4,6,3,1,1,4,0,2,0,2,5,1,8,1,13,1,3,2,4,4,5,0,2,12,5,2,2
3,1,0,5,4,6,3,2,1,0,0,2,0,0,6,1,8,1,15,2,3,2,4,0,5,0,2,13,5,0,2
4,1,0,1,4,6,3,1,1,4,0,2,0,2,15,1,8,1,13,1,3,2,4,2,5,0,2,12,5,2,2


In [40]:
#regress one -hot encoders = problem is fixed!
y = price_log
X = new_cats
regress_this(X, y)

RMSE is 0.2772667338090562
R^2 is 0.5389862522878118
Coefficients:  -0.030160229436939787
Intercepts:  11.129945482501785


In [41]:
# combining all data = best results ! 
ames_all = pd.concat([numerical, ordinal_encoded, new_cats], axis=1)

y = price_log
X = ames_all
regress_this(X, y)

RMSE is 0.15612760308017828
R^2 is 0.8538234430655984
Coefficients:  -0.005352512641609569
Intercepts:  17.842285103915273


# Stretch Goals

- Write a blog post explaining one of today's topics.
- Find a new regression dataset from the UCI machine learning repository and use it to test out your new modeling skillz.
 [ - UCI Machine Learning Repository - Regression Datasets](https://)
- Make a list for yourself of common feature engineering techniques. Browse Kaggle kernels to learn more methods.
- Start studying for tomorrow's topic: Gradient Descent
- Try and make the ultimate model with this dataset. clean as many features as possible, engineer the most sensible features as possible and see how accurate of a prediction you can make. 
- Learn about the "Dummy Variable Trap" and how it applies to linear regression modeling.
- Learning about using linear regression to model time series data